In [44]:
import pandas as pd
import numpy as np
import copy

In [45]:
tanscripts_read= "/home/andre/Documents/trabalho/datasets/Disease_genes/mane/Homo_sapiens.GRCh38.112.chr.gff3"
tanscripts_df = pd.read_csv(tanscripts_read, sep="\t", header=None)
#tanscripts_df

In [3]:
tanscripts_df = tanscripts_df.rename(columns={0: 'Chr', 1: 'Genome', 2: 'Type', 3: 'First_index', 4: 'Last_index', 5: 'Value', 6: 'Signal', 7: 'Unknown', 8: 'Identifiers'})
tanscripts_df

,Chr,Genome,Type,First_index,Last_index,Value,Signal,Unknown,Identifiers
0,1,GRCh38,chromosome,1.0,248956422.0,.,.,.,"ID=chromosome:1;Alias=CM000663.2,chr1,NC_00000..."
1,1,.,biological_region,10469.0,11240.0,1.3e+03,.,.,external_name=oe %3D 0.79;logic_name=cpg
2,1,.,biological_region,10650.0,10657.0,0.999,+,.,logic_name=eponine
3,1,.,biological_region,10655.0,10657.0,0.999,-,.,logic_name=eponine
4,1,.,biological_region,10678.0,10687.0,0.999,+,.,logic_name=eponine
...,...,...,...,...,...,...,...,...,...
3516515,Y,.,biological_region,57215804.0,57215813.0,0.999,-,.,logic_name=eponine
3516516,Y,.,biological_region,57215832.0,57215840.0,0.999,+,.,logic_name=eponine
3516517,Y,.,biological_region,57215833.0,57215842.0,0.999,-,.,logic_name=eponine
3516518,Y,.,biological_region,57215862.0,57215867.0,0.999,+,.,logic_name=eponine


In [4]:
tanscripts_df['Type'].unique()

array(['chromosome', 'biological_region', 'ncRNA_gene', 'lnc_RNA', 'exon',
       nan, 'pseudogene', 'pseudogenic_transcript', 'miRNA', 'gene',
       'mRNA', 'five_prime_UTR', 'CDS', 'three_prime_UTR', 'snRNA',
       'transcript', 'unconfirmed_transcript', 'snoRNA', 'scRNA', 'rRNA',
       'processed_transcript', 'V_gene_segment', 'D_gene_segment',
       'J_gene_segment', 'C_gene_segment', 'tRNA'], dtype=object)

In [5]:
tanscripts_df["Identifiers"][0]

'ID=chromosome:1;Alias=CM000663.2,chr1,NC_000001.11'

## Procurar na coluna "Identifiers" se existem as tags relevantes para fazer uma potêncial filtragem

In [ ]:
tanscripts_df.loc["tag=MANE_Select" in tanscripts_df['Type']]

In [ ]:
#chromosmes = tanscripts_df.loc[tanscripts_df['Type'] == "chromosome"]
#genes = tanscripts_df.loc[tanscripts_df['Type'] == "gene"]
#CDSs = tanscripts_df.loc[tanscripts_df['Type'] == "CDS"]
#UTR3 = tanscripts_df.loc[tanscripts_df['Type'] == "three_prime_UTR"]
#UTR5 = tanscripts_df.loc[tanscripts_df['Type'] == "five_prime_UTR"]
#exons = tanscripts_df.loc[tanscripts_df['Type'] == "exon"]

In [ ]:
#UTR5_overlap = UTR5.loc[UTR5["Identifiers"] == UTR5["Identifiers"][3516301]]

In [7]:
def transcripts_dict_toDF(dictionary):
    transcripts_matrix = []
    concat_list = []
    for value in dictionary.values():
        concat_list.append(value[0])
    return pd.concat(concat_list)

In [27]:
def transcripts_NoOverlaps(element_type = "CDS"):
    element_df = tanscripts_df.loc[tanscripts_df['Type'] == element_type]
    elements_dict = {}
    #elements_dict.update({"header": element_df.columns.tolist()})
    count = 0
    for row in element_df.index:
        identifiers = element_df["Identifiers"][row]
        transcript_id = identifiers.split(";")[0]
        transcript_length = element_df["Last_index"][row] - element_df["First_index"][row]
        if transcript_id not in elements_dict.keys():
        #if elements_dict.get(transcript_id) == None:
            #elements_dict.update({transcript_id: (element_df.values[row], transcript_length)})
            elements_dict.update({transcript_id: (element_df.iloc[[count]], transcript_length)})
        else:
            #print("resolving overlap...")
            if transcript_length > elements_dict[transcript_id][1]:
                #elements_dict.update({transcript_id: (element_df.values[row], transcript_length)})
                elements_dict.update({transcript_id: (element_df.iloc[[count]], transcript_length)})
        count+=1
    return transcripts_dict_toDF(elements_dict)
    #transcripts_matrix = []
    #for value in elements_dict.values():
    #    transcripts_matrix.append(value[0])
    #return elements_dict
    #return pd.DataFrame(elements_dict.values())
    #return pd.DataFrame(transcripts_matrix)

In [ ]:
#fivePrime_UTR_dict = transcripts_NoOverlaps("five_prime_UTR")

In [ ]:
#for value in fivePrime_UTR_dict.values():
#    print(type(value[0]))
#    break

In [ ]:
#UTR5_noOverlaps_df = transcripts_dict_toDF(fivePrime_UTR_dict)

In [ ]:
#UTR5_noOverlaps_df

In [ ]:
#UTR5_noOverlaps_df["Identifiers"][0]

In [8]:
def element_matrix(chromosomes_df, sequence_df):
    """
    The matrix representing all intances of a sequence type available from the original transcripts dataframe
    sequence_df = Genes, CDSs, 5 or 3 prime UTR, Exons
    """
    Chromosomes_matrix = []
    for line in chromosomes_df.index:
        chromosome = chromosomes_df["Chr"][line]
        #print("searching citobands in chromossome: ", chromosome)
        chr_size = int(chromosomes_df["Last_index"][line])
        chr_sequence = sequence_df.loc[sequence_df['Chr'] == chromosome]
        chr_array = np.zeros((chr_size), dtype=int)
        #elements region
        for row in chr_sequence.index:
            chr_array[int(chr_sequence["First_index"][row]) : int(chr_sequence["Last_index"][row])] += 1
                
        
        Chromosomes_matrix.append(chr_array)
    return Chromosomes_matrix

In [9]:
def calc_elementDensity(cytobands_df, region_matrix, col_name="Element_density"):
    #cytobands_df = cytobands_df.drop(cytobands_df[cytobands_df['Chromossome'] == 'chrM'].index)
    densities = []
    for chrom in range(len(cytobands_df["Chromossome"].unique())):             
        chr_cytobands = cytobands_df.loc[cytobands_df['Chromossome'] == cytobands_df["Chromossome"].unique()[chrom]]
        #Calculo da densidade para cada cytoband
        for row in chr_cytobands.index:
            cytoband_array = region_matrix[chrom][int(chr_cytobands["First_index"][row]):int(chr_cytobands["Last_index"][row])]
            dr_length = np.count_nonzero(cytoband_array)
            cytoband_length = chr_cytobands["Size"][row]
            density = dr_length/cytoband_length
            densities.append(round(density, 10))
        
    cytobands_df.insert(len(cytobands_df.columns), col_name, densities, True)

In [10]:
def calc_ExonIntronDensity(cytobands_df, region_matrix, exons_matrix, col_name1="exon_density", col_name2="intron_density"):
    #cytobands_df = cytobands_df.drop(cytobands_df[cytobands_df['Chromossome'] == 'chrM'].index)
    intron_densities = []
    exon_densities = []
    for chrom in range(len(cytobands_df["Chromossome"].unique())):
        #print("Calculating densities in chromosome ", cytobands_df["Chromossome"].unique()[chrom], " ...")
        chr_cytobands = cytobands_df.loc[cytobands_df['Chromossome'] == cytobands_df["Chromossome"].unique()[chrom]]
        #Calculo da densidade para cada cytoband
        for row in chr_cytobands.index:
            ############################ Resolver problema, o conjunto dos exons contem todo o conjunto dos CDSs ###############################
            region_cytoband_array = region_matrix[chrom][int(chr_cytobands["First_index"][row]):int(chr_cytobands["Last_index"][row])]
            exon_cytoband_array = exons_matrix[chrom][int(chr_cytobands["First_index"][row]):int(chr_cytobands["Last_index"][row])]
            #print("cytoband: ", chr_cytobands["Citoband"][row])
            #print("array_regiao: ", region_cytoband_array)
            #print("exon_regiao: ", exon_cytoband_array)
            region_indexes = np.where(region_cytoband_array != 0)[0].tolist()
            exon_indexes = np.where(exon_cytoband_array != 0)[0].tolist()
            #print("number of elements of region: ", len(region_indexes))
            #print("number of exons in region: ", len(np.intersect1d(region_indexes, exon_indexes)))
            #print("exons in CDS region: ", np.intersect1d(region_indexes, exon_indexes))
            #print("introns in CDS region: ", np.setdiff1d(region_indexes, exon_indexes))
            exon_length = len(np.intersect1d(region_indexes, exon_indexes))    #indices que estam na região e são exões
            intron_length = len(np.setdiff1d(region_indexes, exon_indexes))      #indices que estam na região mas não são exões
            #for region_element in region_indexes:
            #    if region_element in exon_indexes:
            #        exon_length+=1
            #    elif region_element not in exon_indexes:
            #        intron_length+=1
            cytoband_length = chr_cytobands["Size"][row]
            exon_density = exon_length/cytoband_length
            intron_density = intron_length/cytoband_length
            exon_densities.append(round(exon_density, 10))
            intron_densities.append(round(intron_density, 10))   #max 6
            
        
    cytobands_df.insert(len(cytobands_df.columns), col_name1, exon_densities, True)
    cytobands_df.insert(len(cytobands_df.columns), col_name2, intron_densities, True)

In [11]:
array1 = [1, 2, 3, 6, 7, 8]
array2 = [2, 3, 4, 7, 8, 9]
print(np.intersect1d(array1, array2))
print(np.setdiff1d(array1, array2))

[2 3 7 8]
[1 6]


In [12]:
def writeNewtxt(file_path, df):
    """Writes a pandas dataframe into txt, columns separated by tabs ("\t")
    """
    f = open(file_path, 'w')
    df_cols = ""
    for col in df.columns[:-1]:
        df_cols += col + "\t"
    df_cols += df.columns.values[-1]+ "\n"
    f.write(str(df_cols))
    for row in df.values:
        df_row = ""
        for col in row[:-1]:
            df_row += str(col) + "\t"
        df_row += str(row[-1])+ "\n"
        f.write(df_row)
    f.close

# Code running

In [13]:
cytoPath_read = "/home/andre/Documents/trabalho/citobands/cytobandFiltered_processed_str_dr.txt"
cytoPath_write = "/home/andre/Documents/trabalho/citobands/cytobandFiltered_processed_str_dr_dg.txt"

cytobands_df = pd.read_csv(cytoPath_read, sep="\t")#, header=None))
cytobands_df2 = pd.read_csv(cytoPath_write, sep="\t")

cytobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404
...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002


In [ ]:
#cytobands_df2

In [14]:
chromosomes = tanscripts_df.loc[tanscripts_df['Type'] == "chromosome"]
#exons = tanscripts_df.loc[tanscripts_df['Type'] == "exon"]

In [ ]:
#exon_matrix = element_matrix(chromosomes, exons)

In [25]:
exons_dict = transcripts_NoOverlaps("exon")
exons_noOverlaps_df = transcripts_dict_toDF(exons_dict)

In [28]:
exon_matrix_noOvrlp = element_matrix(chromosomes, exons_noOverlaps_df)

## Sem filtragem

In [ ]:
def study_transcriptDens(cytobands_df, element_type = "CDS", tanscripts_df = tanscripts_df, chromosomes_df = chromosomes):
    #chromosomes_df = tanscripts_df.loc[tanscripts_df['Type'] == "chromosome"]
    sub_trascripts_df = tanscripts_df.loc[tanscripts_df['Type'] == element_type]
    transcripts_matrix = element_matrix(chromosomes_df, sub_trascripts_df)
    colname = element_type+"s_density"
    calc_elementDensity(cytobands_df, transcripts_matrix, colname)
    writeNewtxt(cytoPath_write, cytobands_df)


In [ ]:
def study_Trcpts_IExons(cytobands_df, element_type = "CDS", tanscripts_df = tanscripts_df, chromosomes_df = chromosomes, exon_matrix = exon_matrix):
    #chromosomes_df = tanscripts_df.loc[tanscripts_df['Type'] == "chromosome"]
    sub_trascripts_df = tanscripts_df.loc[tanscripts_df['Type'] == element_type]
    exons = tanscripts_df.loc[tanscripts_df['Type'] == "exon"]
    print("building matriz...")
    transcripts_matrix = element_matrix(chromosomes_df, sub_trascripts_df)
    #exon_matrix = element_matrix(chromosomes_df, exons)
    print("doing calculations...")
    colname1 = element_type+"Exon_density"
    colname2 = element_type+"Intron_density"
    calc_ExonIntronDensity(cytobands_df, transcripts_matrix, exon_matrix, colname1, colname2)
    writeNewtxt(cytoPath_write, cytobands_df)

## Com filtragem

In [35]:
def study_transcriptDens_noOverlps(cytobands_df, element_type = "CDS", chromosomes_df = chromosomes):
    chromosomes_df = tanscripts_df.loc[tanscripts_df['Type'] == "chromosome"]
    element_noOverlaps_df = transcripts_NoOverlaps(element_type)
    transcript_matrix = element_matrix(chromosomes_df, element_noOverlaps_df)
    colname = element_type+"s_density"
    calc_elementDensity(cytobands_df, transcript_matrix, colname)
    writeNewtxt(cytoPath_write, cytobands_df)

In [34]:
def study_Trcpts_IExons_noOverlps(cytobands_df, element_type = "CDS", chromosomes_df = chromosomes, exon_matrix_noOvrlp = exon_matrix_noOvrlp):
    #chromosomes_df = tanscripts_df.loc[tanscripts_df['Type'] == "chromosome"]
    element_noOverlaps_df = transcripts_NoOverlaps(element_type)
    #exons_dict = transcripts_NoOverlaps("exon")
    #exons_noOverlaps_df = transcripts_dict_toDF(exons_dict)
    print("a fazer a matriz...")
    transcript_matrix = element_matrix(chromosomes_df, element_noOverlaps_df)
    #exon_matrix = element_matrix(chromosmes, exons_noOverlaps_df)
    colname1 = element_type+"Intron_density"
    colname2 = element_type+"Exon_density"
    print("a fazer os calculos...")
    calc_ExonIntronDensity(cytobands_df, transcript_matrix, exon_matrix_noOvrlp, colname1, colname2)
    writeNewtxt(cytoPath_write, cytobands_df)

## CDS density study

In [49]:
#study_transcriptDens(cytobands_df, "CDS")

In [20]:
study_transcriptDens_noOverlps(cytobands_df, "CDS")

### Exon and Intron densities within CDSs (Coding Sequences)

In [ ]:
#study_Trcpts_IExons(cytobands_df, "CDS")

In [23]:
study_Trcpts_IExons_noOverlps(cytobands_df, "CDS")

a fazer a matriz...
a fazer os calculos...


In [24]:
cytobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density,CDSIntron_density,CDSExon_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.016467,0.013392,0.003076
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.006066,0.004490,0.001576
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.010266,0.006903,0.003362
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.005996,0.004821,0.001175
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.009799,0.006866,0.002934
...,...,...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.000917,0.000627,0.000290
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.002263,0.002263,0.000000
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.001096,0.000802,0.000294
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.001369,0.001294,0.000075


## Prime 5 UTR density studies

In [28]:
#study_transcriptDens(cytobands_df, "five_prime_UTR")

In [36]:
study_transcriptDens_noOverlps(cytobands_df, "five_prime_UTR")

### Exon and Intron densities within 5 Prime UTR

In [ ]:
#study_Trcpts_IExons(cytobands_df, "five_prime_UTR")

In [37]:
study_Trcpts_IExons_noOverlps(cytobands_df, "five_prime_UTR")

a fazer a matriz...
a fazer os calculos...


In [38]:
cytobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density,CDSIntron_density,CDSExon_density,five_prime_UTRs_density,five_prime_UTRIntron_density,five_prime_UTRExon_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.016467,0.013392,0.003076,0.010664,0.005135,0.005529
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.006066,0.004490,0.001576,0.003274,0.001761,0.001512
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.010266,0.006903,0.003362,0.005659,0.002073,0.003586
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.005996,0.004821,0.001175,0.003569,0.002070,0.001499
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.009799,0.006866,0.002934,0.004954,0.002176,0.002777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.000917,0.000627,0.000290,0.000927,0.000344,0.000583
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.002263,0.002263,0.000000,0.000346,0.000338,0.000008
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.001096,0.000802,0.000294,0.000540,0.000037,0.000503
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.001369,0.001294,0.000075,0.000652,0.000345,0.000308


## Prime 3 UTR density study

In [33]:
#study_transcriptDens(cytobands_df, "three_prime_UTR")

In [39]:
study_transcriptDens_noOverlps(cytobands_df, "three_prime_UTR")

## Exon and Intron densities within 3 Prime UTR

In [ ]:
#study_Trcpts_IExons(cytobands_df, "three_prime_UTR")

In [40]:
study_Trcpts_IExons_noOverlps(cytobands_df, "three_prime_UTR")

a fazer a matriz...
a fazer os calculos...


In [41]:
cytobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density,CDSIntron_density,CDSExon_density,five_prime_UTRs_density,five_prime_UTRIntron_density,five_prime_UTRExon_density,three_prime_UTRs_density,three_prime_UTRIntron_density,three_prime_UTRExon_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.016467,0.013392,0.003076,0.010664,0.005135,0.005529,0.026310,0.025853,0.000456
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.006066,0.004490,0.001576,0.003274,0.001761,0.001512,0.019776,0.019406,0.000370
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.010266,0.006903,0.003362,0.005659,0.002073,0.003586,0.025848,0.025715,0.000133
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.005996,0.004821,0.001175,0.003569,0.002070,0.001499,0.015875,0.015746,0.000129
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.009799,0.006866,0.002934,0.004954,0.002176,0.002777,0.029553,0.028516,0.001037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.000917,0.000627,0.000290,0.000927,0.000344,0.000583,0.002534,0.002459,0.000075
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.002263,0.002263,0.000000,0.000346,0.000338,0.000008,0.000937,0.000635,0.000302
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.001096,0.000802,0.000294,0.000540,0.000037,0.000503,0.002733,0.002733,0.000000
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.001369,0.001294,0.000075,0.000652,0.000345,0.000308,0.002016,0.001749,0.000267


## Exons density study

In [42]:
study_transcriptDens_noOverlps(cytobands_df, "exon")

In [43]:
cytobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density,CDSIntron_density,CDSExon_density,five_prime_UTRs_density,five_prime_UTRIntron_density,five_prime_UTRExon_density,three_prime_UTRs_density,three_prime_UTRIntron_density,three_prime_UTRExon_density,exons_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.016467,0.013392,0.003076,0.010664,0.005135,0.005529,0.026310,0.025853,0.000456,0.133749
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.006066,0.004490,0.001576,0.003274,0.001761,0.001512,0.019776,0.019406,0.000370,0.060012
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.010266,0.006903,0.003362,0.005659,0.002073,0.003586,0.025848,0.025715,0.000133,0.056062
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.005996,0.004821,0.001175,0.003569,0.002070,0.001499,0.015875,0.015746,0.000129,0.034280
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.009799,0.006866,0.002934,0.004954,0.002176,0.002777,0.029553,0.028516,0.001037,0.066018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.000917,0.000627,0.000290,0.000927,0.000344,0.000583,0.002534,0.002459,0.000075,0.009953
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.002263,0.002263,0.000000,0.000346,0.000338,0.000008,0.000937,0.000635,0.000302,0.033020
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.001096,0.000802,0.000294,0.000540,0.000037,0.000503,0.002733,0.002733,0.000000,0.021283
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.001369,0.001294,0.000075,0.000652,0.000345,0.000308,0.002016,0.001749,0.000267,0.022278


## Resultados sem usar filtros, o conjunto dos exões cobre completamente as regiões estudadas

In [46]:
#Anterior resultado sem usar filtros ###cytobands_df

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density,CDSIntron_density,CDSExon_density,5P_UTR_density,UTR5Intron_density,UTR5Exon_density,3P_UTR_density,UTR3Intron_density,UTR3Exon_density,Exons_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.038,0.022,0.015,0.033,0.022,0.021,0.012,0.025,0.019,0.171
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.016,0.011,0.005,0.022,0.010,0.014,0.004,0.013,0.006,0.079
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.024,0.010,0.008,0.034,0.009,0.020,0.006,0.012,0.010,0.082
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.013,0.004,0.002,0.018,0.003,0.010,0.005,0.004,0.004,0.047
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.030,0.010,0.008,0.036,0.010,0.019,0.005,0.014,0.010,0.093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.004,0.001,0.001,0.003,0.001,0.003,0.001,0.001,0.003,0.016
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.002,0.004,0.004,0.001,0.004,0.003,0.000,0.004,0.002,0.045
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.005,0.004,0.002,0.004,0.004,0.002,0.001,0.005,0.002,0.036
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.003,0.002,0.000,0.002,0.003,0.001,0.001,0.003,0.001,0.034


In [43]:
#cytobands_df.values[cytobands_df.index[70]]

In [12]:
def sumOf_columns(df, colums=["CDSIntron_density", "UTR5Intron_density", "UTR3Intron_density"], newColName = "Introns_density"):
    """
    colums is a list of string
    """
    #cytobands_df = cytobands_df.drop(cytobands_df[cytobands_df['Chromossome'] == 'chrM'].index)
    new_densites = []
    for row in df.index:
        density =0
        for column in colums:
            density += df[column][row]
        new_densites.append(round(density, 3))
        
    df.insert(len(df.columns), newColName, new_densites, True)

In [14]:
#sumOf_columns(cytobands_df2)
cytobands_df2

,Chromossome,First_index,Last_index,Citoband,Unkown,Size,STR_Density,DR_density,CDSs_density,CDSIntron_density,CDSExon_density,5P_UTR_density,UTR5Intron_density,UTR5Exon_density,3P_UTR_density,UTR3Intron_density,UTR3Exon_density,Exons_density,Introns_density
0,chr1,0,2299999,p36.33,gneg,2299999,0.088,0.326,0.038,0.022,0.015,0.033,0.022,0.021,0.012,0.025,0.019,0.171,0.069
1,chr1,2300000,5299999,p36.32,gpos25,2999999,0.084,0.282,0.016,0.011,0.005,0.022,0.010,0.014,0.004,0.013,0.006,0.079,0.034
2,chr1,5300000,7099999,p36.31,gneg,1799999,0.056,0.275,0.024,0.010,0.008,0.034,0.009,0.020,0.006,0.012,0.010,0.082,0.031
3,chr1,7100000,9099999,p36.23,gpos25,1999999,0.049,0.405,0.013,0.004,0.002,0.018,0.003,0.010,0.005,0.004,0.004,0.047,0.011
4,chr1,9100000,12499999,p36.22,gneg,3399999,0.056,0.404,0.030,0.010,0.008,0.036,0.010,0.019,0.005,0.014,0.010,0.093,0.034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,chrY,12400000,17099999,q11.221,gpos50,4699999,0.037,0.015,0.004,0.001,0.001,0.003,0.001,0.003,0.001,0.001,0.003,0.016,0.003
859,chrY,17100000,19599999,q11.222,gneg,2499999,0.062,0.013,0.002,0.004,0.004,0.001,0.004,0.003,0.000,0.004,0.002,0.045,0.012
860,chrY,19600000,23799999,q11.223,gpos50,4199999,0.082,0.006,0.005,0.004,0.002,0.004,0.004,0.002,0.001,0.005,0.002,0.036,0.013
861,chrY,23800000,26599999,q11.23,gneg,2799999,0.026,0.002,0.003,0.002,0.000,0.002,0.003,0.001,0.001,0.003,0.001,0.034,0.008


In [17]:
writeNewtxt(cytoPath_write, cytobands_df2)

In [24]:
CDSs = tanscripts_df.loc[tanscripts_df['Type'] == "CDS"]

In [26]:
CDSs["Identifiers"][99]

'ID=CDS:ENSP00000493376;Parent=transcript:ENST00000641515;protein_id=ENSP00000493376'